In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
!pip install transforms3d

     |████████████████████████████████| 71kB 6.6MB/s 
  Created wheel for transforms3d: filename=transforms3d-0.3.1-cp36-none-any.whl size=59374 sha256=4c53c40f717e445484b5bd4408fe8078c91757953608fc60a6a6979875c6bb44
  Stored in directory: /root/.cache/pip/wheels/3d/3c/84/28d36677f3c760c048bd02b5a547ea0c4027770cc9cdb9af1e
Successfully built transforms3d


In [19]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal, spatial, stats
from fastdtw import fastdtw
from scipy.signal import find_peaks
import pywt
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from scipy.interpolate import CubicSpline      # for warping
from transforms3d.axangles import axangle2mat  # for rotation
from keras.layers import Input, Dense, Conv1D, Conv2D, MaxPooling2D, MaxPooling1D, Flatten, Dropout, BatchNormalization, Reshape, TimeDistributed, LSTM
from keras.models import Model
from sklearn.preprocessing import OneHotEncoder
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from keras.metrics import Accuracy, Precision, Recall
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import Sequential

In [4]:
os.chdir('drive/My Drive/Weave/Earables Project/Final Experiments/')

In [5]:
data = pickle.load(open('data/processed_df.pkl', 'rb'))
fold_idx = pickle.load(open('data/5_fold_idx.pkl', 'rb'))

In [7]:
test_fold_i = 3
train_idx, test_idx =[], []
for i, idx in enumerate(fold_idx):
    if i == test_fold_i:
        test_idx.append(idx)
    else:
        train_idx.append(idx)
train_idx = np.concatenate(train_idx)
test_idx = np.concatenate(test_idx)
print('train: {}, test: {}'.format(len(train_idx), len(test_idx)))
X_train = data.loc[train_idx].iloc[:,:-3].to_numpy()
y_train = data.loc[train_idx].iloc[:,-3:-2]
X_test = data.loc[test_idx].iloc[:,:-3].to_numpy()
y_test = data.loc[test_idx].iloc[:,-3:-2]
ohe = OneHotEncoder()
y_train = ohe.fit_transform(y_train).toarray()
y_test = ohe.transform(y_test).toarray()

train: 14984, test: 3746


In [8]:
nb_classes = y_train.shape[1]
print('num. of classes: {}'.format(nb_classes))

num. of classes: 32


In [9]:
class Data_Generator(keras.utils.Sequence):
    
    def __init__(self, data_x, labels, batch_size, augment=True) :
        self.data_x = data_x
        self.labels = labels
        self.batch_size = batch_size
        self.augment = augment
    
    def __len__(self):
        return (np.ceil(len(self.data_x) / float(self.batch_size))).astype(np.int)
  
    def __getitem__(self, idx) :
        x = self.data_x[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
        
        batch_x = np.ones((x.shape[0], 100, x.shape[1]))

        for i in range(len(x)):
            if self.augment:
                aug = np.random.choice(range(4))
            else:
                aug = 3
            #print(aug)
            temp = np.array([np.array(a) for a in x[i]])
            if aug == 0:
                batch_x[i] = np.hstack([self.DA_TimeWarp(temp[3*j:3*(j+1)].T) for j in range(len(temp)//3)])
            elif aug == 1:
                batch_x[i] = np.hstack([self.DA_MagWarp(temp[3*j:3*(j+1)].T, 0.2) for j in range(len(temp)//3)])
            elif aug == 2:
                batch_x[i] = np.hstack([self.DA_Rotation(temp[3*j:3*(j+1)].T) for j in range(len(temp)//3)])
            else:
                batch_x[i] = temp.T

        return np.array(batch_x), np.array(batch_y)

    def DistortTimesteps(self, X, sigma=0.2):
        tt = self.GenerateRandomCurves(X, sigma) # Regard these samples aroun 1 as time intervals
        tt_cum = np.cumsum(tt, axis=0)        # Add intervals to make a cumulative graph
        # Make the last value to have X.shape[0]
        t_scale = [(X.shape[0]-1)/tt_cum[-1,0],(X.shape[0]-1)/tt_cum[-1,1],(X.shape[0]-1)/tt_cum[-1,2]]
        tt_cum[:,0] = tt_cum[:,0]*t_scale[0]
        tt_cum[:,1] = tt_cum[:,1]*t_scale[1]
        tt_cum[:,2] = tt_cum[:,2]*t_scale[2]
        return tt_cum

    def DA_TimeWarp(self, X, sigma=0.2):
        tt_new = self.DistortTimesteps(X, sigma)
        X_new = np.zeros(X.shape)
        x_range = np.arange(X.shape[0])
        X_new[:,0] = np.interp(x_range, tt_new[:,0], X[:,0])
        X_new[:,1] = np.interp(x_range, tt_new[:,1], X[:,1])
        X_new[:,2] = np.interp(x_range, tt_new[:,2], X[:,2])
        return X_new

    def DA_Rotation(self, X):
        axis = np.random.uniform(low=-1, high=1, size=X.shape[1])
        angle = np.random.uniform(low=-np.pi, high=np.pi)
        return np.matmul(X , axangle2mat(axis,angle))

    def GenerateRandomCurves(self, X, sigma=0.2, knot=4):
        xx = (np.ones((X.shape[1],1))*(np.arange(0,X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
        yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, X.shape[1]))
        x_range = np.arange(X.shape[0])
        cs_x = CubicSpline(xx[:,0], yy[:,0])
        cs_y = CubicSpline(xx[:,1], yy[:,1])
        cs_z = CubicSpline(xx[:,2], yy[:,2])
        return np.array([cs_x(x_range),cs_y(x_range),cs_z(x_range)]).transpose()
    
    def DA_MagWarp(self, X, sigma):
        return X * self.GenerateRandomCurves(X, sigma)

In [58]:
def build_model(input_shape, nb_classes):
    model = Sequential(name = 'lstm_12ch')
    model.add(Input(shape=input_shape, name='input'))
    #model.add(Reshape((input_shape[0], input_shape[1])))
    model.add(LSTM(256, return_sequences=True))
    model.add(LSTM(512))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(nb_classes, activation = 'softmax'))

    return model

In [59]:
input_shape = (100, 12)
model = build_model(input_shape, nb_classes)
model.summary()

Model: "lstm_12ch"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 100, 256)          275456    
_________________________________________________________________
lstm_15 (LSTM)               (None, 512)               1574912   
_________________________________________________________________
dense_27 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_28 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_29 (Dense)             (None, 32)                4128      
Total params: 2,018,720
Trainable params: 2,018,720
Non-trainable params: 0
_________________________________________________________________


In [60]:
batch_size = 32
train_gen = Data_Generator(X_train, y_train, batch_size)
test_gen = Data_Generator(X_test, y_test, batch_size)

In [61]:
model_name = 'lstm_12ch_softmax'
# optimizer = Adam(learning_rate=0.001)
# loss = BinaryCrossentropy()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
checkpoint = ModelCheckpoint('checkpoint/{}.h5'.format(model_name), monitor='val_loss', verbose=1, mode='min',save_best_only=True)
earlystopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

In [ ]:
history = model.fit(train_gen,
                    validation_data=test_gen,
                    epochs=100,
                    workers=4,
                    callbacks=[checkpoint, earlystopping],
                    shuffle=True)